In [2]:
from google.colab import drive
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import EarlyStoppingCallback
import torch.nn as nn

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
finance_data = '/content/drive/MyDrive/finance_data.csv'
df = pd.read_csv(finance_data)

In [5]:
le = LabelEncoder()
df['Sentiment'] = le.fit_transform(df['Sentiment'])

In [6]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def tokenize_function(sentences):
    return tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

In [8]:
X = df['Sentence']
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
train_encodings = tokenize_function(X_train.tolist())
test_encodings = tokenize_function(X_test.tolist())

In [10]:
train_labels = torch.tensor(y_train.tolist())
test_labels = torch.tensor(y_test.tolist())

In [11]:
class FinancialSentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
train_dataset = FinancialSentimentDataset(train_encodings, train_labels)
test_dataset = FinancialSentimentDataset(test_encodings, test_labels)

In [13]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    learning_rate=2e-5
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.868300,0.774329
2,0.472400,0.383136
3,0.340500,0.395716
4,0.260700,0.464182
5,0.277000,0.479326


TrainOutput(global_step=735, training_loss=0.48572906166517815, metrics={'train_runtime': 130.6141, 'train_samples_per_second': 357.772, 'train_steps_per_second': 11.255, 'total_flos': 1200711915081000.0, 'train_loss': 0.48572906166517815, 'epoch': 5.0})

In [16]:
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

In [17]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

In [18]:
print(f'RoBERTa Accuracy: {accuracy}')
print(report)

RoBERTa Accuracy: 0.8212147134302823
              precision    recall  f1-score   support

    negative       0.57      0.40      0.47       172
     neutral       0.83      0.89      0.86       626
    positive       0.88      0.89      0.89       371

    accuracy                           0.82      1169
   macro avg       0.76      0.73      0.74      1169
weighted avg       0.81      0.82      0.81      1169



In [19]:
def categorical_focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):

    y_pred = torch.clamp(y_pred, 1e-7, 1 - 1e-7)

    ce_loss = -y_true * torch.log(y_pred)

    focal_loss = alpha * torch.pow(1 - y_pred, gamma) * ce_loss

    return torch.mean(torch.sum(focal_loss, axis=-1))

In [20]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        num_classes = logits.size(-1)
        labels_one_hot = nn.functional.one_hot(labels, num_classes=num_classes).float()

        loss = categorical_focal_loss(labels_one_hot, torch.softmax(logits, dim=-1))

        return (loss, outputs) if return_outputs else loss

In [21]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.021300,0.037597
2,0.022100,0.041318
3,0.022400,0.040413
4,0.014900,0.039399


TrainOutput(global_step=588, training_loss=0.02416182152268027, metrics={'train_runtime': 95.4509, 'train_samples_per_second': 489.571, 'train_steps_per_second': 15.401, 'total_flos': 960569532064800.0, 'train_loss': 0.02416182152268027, 'epoch': 4.0})

In [23]:
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

In [24]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

In [25]:
print(f'RoBERTa with Focal Loss Accuracy: {accuracy}')
print(report)

RoBERTa with Focal Loss Accuracy: 0.8092386655260907
              precision    recall  f1-score   support

    negative       0.54      0.76      0.63       172
     neutral       0.92      0.75      0.83       626
    positive       0.84      0.93      0.88       371

    accuracy                           0.81      1169
   macro avg       0.76      0.81      0.78      1169
weighted avg       0.84      0.81      0.81      1169

